In [ ]:
import time

import numpy as np
from manipulation.station import MakeHardwareStation, MakeHardwareStationInterface, load_scenario

from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    ApplyMultibodyPlantConfig,    
    DiagramBuilder,
    GcsTrajectoryOptimization,
    HPolyhedron,
    InverseKinematics,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    ModelDirectives,
    MultibodyPlant,
    Parser,
    PiecewisePolynomial,  
    ProcessModelDirectives,    
    Point,
    Solve,
    StartMeshcat,
    TrajectorySource,
    Simulator,
)
from manipulation.utils import FindResource

rng = np.random.default_rng()

In [ ]:
meshcat = StartMeshcat()

In [ ]:
scenario = load_scenario(filename=FindResource("scenarios/gcs_kuka_demo0.yaml"), scenario_name="Demo")

This code runs in one of two modes:
1. When `hardware=False`, then everything runs in a single process
2. When `hardware=True`, then it connects via LCM to the robot driver (or a station simulation)

To run the hardware simulator in a different process, use e.g.
```
python3 drake/examples/hardware_sim/hardware_sim.py --scenario_file=/Users/russt/manipulation/manipulation/scenarios/gcs_kuka_demo0.yaml --scenario_name=Demo 
```

In [ ]:
def PublishPositionTrajectory(
    scenario, trajectory, time_step=1.0 / 33.0
):
    """
    Args:
        trajectory: A Trajectory instance.
    """
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
    ApplyMultibodyPlantConfig(scenario.plant_config, plant)
    ProcessModelDirectives(
        directives=ModelDirectives(directives=scenario.directives),
        plant=plant)
    plant.Finalize()

    params = MeshcatVisualizerParams()
    params.prefix = "planned trajectory"
    params.enable_alpha_slider = True
    MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat, params)
    meshcat.SetSliderValue("planned trajectory α", 0.5)
    diagram = builder.Build()
    root_context = diagram.CreateDefaultContext()

    plant_context = plant.GetMyContextFromRoot(root_context)

    meshcat.StartRecording(set_visualizations_while_recording=False)

    for t in np.append(
        np.arange(trajectory.start_time(), trajectory.end_time(), time_step),
        trajectory.end_time(),
    ):
        root_context.SetTime(t)
        plant.SetPositions(plant_context, trajectory.value(t))
        diagram.ForcedPublish(root_context)

    meshcat.StopRecording()
    meshcat.PublishRecording()
    meshcat.DeleteSlider("planned trajectory α")

def MyInverseKinematics(plant, p_WE):
    context = plant.CreateDefaultContext()
    # E = ee_body.body_frame()
    E = plant.GetBodyByName("iiwa_link_7").body_frame()

    ik = InverseKinematics(plant, context)

    ik.AddPositionConstraint(
        E, [0, 0, 0.1], plant.world_frame(), p_WE, p_WE
    )

    prog = ik.get_mutable_prog()
    q = ik.q()

    q0 = plant.GetPositions(context)
    prog.AddQuadraticErrorCost(np.identity(len(q)), q0, q)
    prog.SetInitialGuess(q, q0)
    result = Solve(ik.prog())
    if not result.is_success():
        print("IK failed")
        return None
    plant.SetPositions(context, result.GetSolution(q))
    return result.GetSolution(q)

def plan_and_maybe_execute(scenario, meshcat, hardware=False):
    meshcat.Delete()

    builder = DiagramBuilder()

    if hardware:
            station = builder.AddNamedSystem(
        "station",
        MakeHardwareStationInterface(
            scenario,
            meshcat=meshcat,
        ),
    )
    else:
        station = builder.AddSystem(
            MakeHardwareStation(
                scenario,
                meshcat=meshcat,
            ),
        )
    
    # wsg_position = builder.AddSystem(ConstantVectorSource([WsgPositions.OPEN.value]))
    # builder.Connect(
    #    wsg_position.get_output_port(), station.GetInputPort("wsg_position")
    # )
    plant = MultibodyPlant(scenario.plant_config.time_step)
    ApplyMultibodyPlantConfig(scenario.plant_config, plant)
    ProcessModelDirectives(
        directives=ModelDirectives(directives=scenario.directives),
        plant=plant)
    plant.Finalize()

    num_positions = plant.num_positions()
    default_positions = plant.GetPositions(plant.CreateDefaultContext())
    position_trajectory = builder.AddNamedSystem(
        "position_trajectory", TrajectorySource(PiecewisePolynomial(default_positions))
    )
    builder.Connect(
        position_trajectory.get_output_port(), station.GetInputPort("iiwa_position")
    )

    builder.ExportOutput(
        station.GetOutputPort("iiwa_position_commanded"), "iiwa_position_commanded"
    )

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    diagram.ExecuteInitializationEvents(context)

    gcs = GcsTrajectoryOptimization(plant.num_positions())

    workspace = gcs.AddRegions(
        [
            HPolyhedron.MakeBox(
                plant.GetPositionLowerLimits(), plant.GetPositionUpperLimits()
            )
        ],
        5,
        1,
        60,
    )

    q_start = diagram.GetOutputPort("iiwa_position_commanded").Eval(context)
    print(q_start)
    p_WE = rng.uniform([0.5, -0.2, 0.5], [0.7, 0.2, 0.8])
    q_goal = MyInverseKinematics(plant, p_WE)

    # Set non-zero h_min for start and goal to enforce zero velocity.
    start = gcs.AddRegions([Point(q_start)], order=1, h_min=0.1)
    goal = gcs.AddRegions([Point(q_goal)], order=1, h_min=0.1)
    goal.AddVelocityBounds([0]*num_positions, [0]*num_positions)
    gcs.AddEdges(start, workspace)
    gcs.AddEdges(workspace, goal)
    gcs.AddTimeCost()
    gcs.AddPathLengthCost()
    gcs.AddVelocityBounds(
        0.25*plant.GetVelocityLowerLimits(), 0.25*plant.GetVelocityUpperLimits()
    )
    traj, result = gcs.SolvePath(start, goal)

    PublishPositionTrajectory(scenario, traj)

    print("Review trajectory in meshcat then hit 'Ok to execute' or 'Cancel'")
    meshcat.AddButton("Ok to execute")
    meshcat.AddButton("Cancel")
    while (
        meshcat.GetButtonClicks("Ok to execute") < 1
        and meshcat.GetButtonClicks("Cancel") < 1
    ):
        time.sleep(0.1)
    execute = meshcat.GetButtonClicks("Ok to execute") > 0
    meshcat.DeleteButton("Ok to execute")
    meshcat.DeleteButton("Cancel")

    if execute:
        print("Executing trajectory...")
        position_trajectory = diagram.GetSubsystemByName("position_trajectory")
        position_trajectory.UpdateTrajectory(traj)
        simulator = Simulator(diagram)
        simulator.set_target_realtime_rate(1.0)
        simulator.AdvanceTo(traj.end_time())


plan_and_maybe_execute(scenario, meshcat, hardware=False)